In [2]:
    # 이미지 링크 가져오기
#     img_src = item.find_element_by_class_name('product_img').get_attribute('src')
    
    # 배경 색상 가져오기
#     bg_color = item.find_element_by_class_name('product').get_attribute('style')


In [63]:
import warnings
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time
import random
import datetime
from tqdm.auto import tqdm, trange
import numpy as np
import pandas as pd

# 크롤링

In [ ]:
# 경고문 제어
warnings.filterwarnings(action='ignore')

# 자동 날짜 갱신을 위한 변수
date = datetime.datetime.now()

# 크롬드라이버 설정
driver = webdriver.Chrome(ChromeDriverManager().install())

# 크림 페이지 주소 변수 저장
url = "http://kream.co.kr/search?sort=popular&per_page=20"

# 불러오기
driver.get(url)

In [ ]:
# 모든 페이지 불러오기 위해 스크롤 내리기
for i in range(10):
    time.sleep(1)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
print("페이지 스크롤 완료")

In [ ]:
product_infos = []

# 페이지 내의 상품 리스트를 가져옴
items = driver.find_element_by_class_name('search_result_list')
item_list = items.find_elements_by_class_name('search_result_item')

for item in item_list:
    # 제목, 가격 등 기본 정보 가져오기
    info = item.find_element_by_class_name('item_inner')
    text = info.text.split("\n")
    
    info2 = item.find_element_by_class_name('interest_figure')
    interest = info2.text.split("\n")
    
    #인덱싱으로 브랜드, 이름, 리셀가격, 빠른배송 여부, 좋아요, 위시 긁어오기
    if len(text) == 6:
        brand = text[0]
        en_name = text[1]
        kor_name = text[2]
        re_price = text[3][:-1]
        speed_deli = 1
    else:
        brand = text[0]
        en_name = text[1]
        kor_name = text[2]
        re_price = text[3][:-1]
        speed_deli = 0
    wish = interest[0]
    review = interest[1]
    
    # 상품 세부 페이지 link 가져오기
    link = info.get_attribute('href')

    # 저장
    item_dict = {
        "brand": brand,
        "en_name": en_name,
        "kor_name": kor_name,
        "re_price": re_price,
        "speed_deli": speed_deli,
        "wish": wish,
        "review": review,
        'link': link
    }

    product_infos.append(item_dict)

print("크롤링 완료")
df = pd.DataFrame(product_infos)
df.to_csv("process_1.csv", index = False)

In [ ]:
product_detail = []
pro_num = 0
num = 0

for i in trange(len(df)):
    num+=1
    while pro_num < num:
        if pro_num == 361:
            break
        try:
            number = df["link"][i].split("/")[4]
            url = f"https://kream.co.kr/products/{number}"
            time.sleep(random.randint(1, 2))
            driver.get(url)

            product_items = driver.find_element_by_class_name('detail_product')
            items = product_items.text.split("\n")

            product_dict = {
                "link" : url,
                "model" : items[1],
                "release" : items[3],
                "color" : items[5],
                "price" : items[7]
            }
            product_detail.append(product_dict)
            pro_num += 1
        except:
            print(f"{i+1}번 크롤링 재시작")
            driver.refresh()

df1 = pd.DataFrame(product_detail)

# merge 및 전처리

In [15]:
def preprocessing(date):
    # 최종 merge
    total = pd.merge(df, df1, on="link")
    # 상품번호 분리
    total["product_num"] = total["link"].apply(lambda x: x.split("/")[4])
    # 컬럼 재정렬
    total = total[["brand", "en_name", "kor_name", "product_num", "model", "release", "color", 
                   "price", "re_price", "wish", "review", "link"]]
    # 저장
    total.to_csv(f"{date}_kream.csv", index = False)
    return total

In [ ]:
# 전처리 후 저장하는 함수 
preprocessing(date.strftime("%Y%m%d"))